## Fairness in AI

### Fine tuning Whisper-Small with AfriSpeech-200 South African English Accent Data.
Apr 16th 2024

reference:https://huggingface.co/blog/fine-tune-whisper?source=post_page-----976164a5eac8--------------------------------

afriSpeech-2000: https://huggingface.co/datasets/tobiolatunji/afrispeech-200

### Preparing Environment

In [ ]:
!pip install datasets jiwer
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install -U accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperProcessor
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor
from jiwer import wer
from datasets import load_dataset, DatasetDict
import torch
from evaluate import load
from transformers import WhisperTokenizer
from datasets import Audio, load_dataset
import torch
from transformers import WhisperForConditionalGeneration
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
import librosa
from transformers import WhisperProcessor
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preprocessing AfriSpeech-200 South African English Accent Data

In [ ]:
from datasets import load_dataset, DatasetDict

afrispeech3 = DatasetDict()
afrispeech3['train'] = load_dataset("tobiolatunji/afrispeech-200", "south-african-english", split="train")
afrispeech3["test"] = load_dataset("tobiolatunji/afrispeech-200", "south-african-english", split="test")

print(afrispeech3)
afrispeech3 = afrispeech3.remove_columns(['speaker_id', 'path', 'audio_id', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for tobiolatunji/afrispeech-200 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tobiolatunji/afrispeech-200
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release o

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 114it [00:00, 32408.20it/s]


Generating test split: 0 examples [00:00, ? examples/s]


Reading metadata...: 5it [00:00, 21334.20it/s]


DatasetDict({
    train: Dataset({
        features: ['speaker_id', 'path', 'audio_id', 'audio', 'transcript', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'],
        num_rows: 114
    })
    test: Dataset({
        features: ['speaker_id', 'path', 'audio_id', 'audio', 'transcript', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'],
        num_rows: 5
    })
})


In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="en", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
input_str = afrispeech3["train"][0]["transcript"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Jide Sow does not want to get re-hospitalized in the future given past experience on 13/08/2026

Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>Jide Sow does not want to get re-hospitalized in the future given past experience on 13/08/2026
<|endoftext|>
Decoded w/out special: Jide Sow does not want to get re-hospitalized in the future given past experience on 13/08/2026

Are equal:             True


In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="en", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
afrispeech3 = afrispeech3.cast_column("audio", Audio(sampling_rate=16000))

print(afrispeech3["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/101055b5a874ac7622614be272e1ecdcc11e133c718dc8a662762a6cc85f69da/7f75e28a-a56a-45b7-b44f-2eac8d5aee2f/3bb362dd76709befbd34cbdbcd876304.wav', 'array': array([-0.00254716, -0.00529757, -0.00501746, ...,  0.00102808,
        0.00149547,  0.00170032]), 'sampling_rate': 16000}, 'transcript': 'Jide Sow does not want to get re-hospitalized in the future given past experience on 13/08/2026\n'}


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcript"]).input_ids
    return batch

afrispeech3 = afrispeech3.map(prepare_dataset, remove_columns=afrispeech3.column_names["train"], num_proc=4)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/114 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Model Training

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.generation_config.language = "en"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args4 = Seq2SeqTrainingArguments(
    output_dir="/content/",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=5e-6,
    warmup_steps=200,
    max_steps=400,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=50,
    eval_steps=25,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer4 = Seq2SeqTrainer(
    args=training_args4,
    model=model,
    train_dataset=afrispeech3["train"],
    eval_dataset=afrispeech3["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer4.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
25,2.981900,1.942647,33.333333
50,2.389900,1.410284,27.956989
75,1.577300,1.032623,27.956989
100,0.998700,0.839739,27.956989
125,0.633600,0.803855,26.881720
150,0.465800,0.766345,25.806452
175,0.337800,0.733365,25.806452
200,0.250000,0.731331,25.806452
225,0.153000,0.669172,23.655914
250,0.051900,0.595193,24.731183


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint

TrainOutput(global_step=400, training_loss=0.6176994832977653, metrics={'train_runtime': 1719.2663, 'train_samples_per_second': 3.723, 'train_steps_per_second': 0.233, 'total_flos': 1.644936781824e+18, 'train_loss': 0.6176994832977653, 'epoch': 50.0})